# Fixed Effects Estimation (Example II)

### Intro and objectives


### In this lab you will learn:
1. examples of fixed effects estimation
2. how to fit fixed effects models in Python


## What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you need to fit fixed effects models
* Worked Examples
* How to interpret the results obtained

In [1]:
!pip install wooldridge
!pip install linearmodels
import wooldridge as woo
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import linearmodels as plm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 36.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 KB 5.9 MB/s eta 0:00:00


# Example. Has the Return to Education Changed over Time?

#### The data in WAGEPAN are from Vella and Verbeek (1998). Each of the 545 men in the sample worked in every year from 1980 through 1987. Some variables in the data set change over time: experience, marital status, and union status are the three important ones. Other variables do not change: race and education are the key examples. 

#### If we use fixed effects, we cannot include race, education, or experience in the equation as they would be eliminated in the transformed model.


#### However, we can include interactions of educ with year dummies for 1981 through 1987 to test whether the return to education was constant over this time period. We use log(wage) as the dependent variable, dummy variables for marital and union status, a full set of year dummies, and the interaction terms: d81\*educ, d82\*educ, . . . , d87\*educ.

#### We assume a model of the following form:

$log(wage)=\beta_0+\beta_1*married+\beta_2*union+\delta_0*y_{1980}*education+…+\delta_8*y_{1987}*education$






In [8]:
wagepan = woo.dataWoo('wagepan')
wagepan = wagepan.set_index(['nr', 'year'], drop=False)

In [7]:
jtrain.head()

,nr,year,agric,black,bus,construc,ent,exper,fin,hisp,...,union,lwage,d81,d82,d83,d84,d85,d86,d87,expersq
0,13,1980,0,0,1,0,0,1,0,0,...,0,1.197540,0,0,0,0,0,0,0,1
1,13,1981,0,0,0,0,0,2,0,0,...,1,1.853060,1,0,0,0,0,0,0,4
2,13,1982,0,0,1,0,0,3,0,0,...,0,1.344462,0,1,0,0,0,0,0,9
3,13,1983,0,0,1,0,0,4,0,0,...,0,1.433213,0,0,1,0,0,0,0,16
4,13,1984,0,0,0,0,0,5,0,0,...,0,1.568125,0,0,0,1,0,0,0,25


In [22]:
jtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 44 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   nr        4360 non-null   int64  
 1   year      4360 non-null   int64  
 2   agric     4360 non-null   int64  
 3   black     4360 non-null   int64  
 4   bus       4360 non-null   int64  
 5   construc  4360 non-null   int64  
 6   ent       4360 non-null   int64  
 7   exper     4360 non-null   int64  
 8   fin       4360 non-null   int64  
 9   hisp      4360 non-null   int64  
 10  poorhlth  4360 non-null   int64  
 11  hours     4360 non-null   int64  
 12  manuf     4360 non-null   int64  
 13  married   4360 non-null   int64  
 14  min       4360 non-null   int64  
 15  nrthcen   4360 non-null   int64  
 16  nrtheast  4360 non-null   int64  
 17  occ1      4360 non-null   int64  
 18  occ2      4360 non-null   int64  
 19  occ3      4360 non-null   int64  
 20  occ4      4360 non-null   int6

In [15]:
jtrain[['nr','year','black','bus','construc','exper','hisp','union','lwage','d81','d82','d83','expersq','married']].head(20)

,nr,year,black,bus,construc,exper,hisp,union,lwage,d81,d82,d83,expersq,married
0,13,1980,0,1,0,1,0,0,1.197540,0,0,0,1,0
1,13,1981,0,0,0,2,0,1,1.853060,1,0,0,4,0
2,13,1982,0,1,0,3,0,0,1.344462,0,1,0,9,0
3,13,1983,0,1,0,4,0,0,1.433213,0,0,1,16,0
4,13,1984,0,0,0,5,0,0,1.568125,0,0,0,25,0
5,13,1985,0,1,0,6,0,0,1.699891,0,0,0,36,0
6,13,1986,0,1,0,7,0,0,-0.720263,0,0,0,49,0
7,13,1987,0,1,0,8,0,0,1.669188,0,0,0,64,0
8,17,1980,0,0,0,4,0,0,1.675962,0,0,0,16,0
9,17,1981,0,0,0,5,0,0,1.518398,1,0,0,25,0


In [20]:
# FE model estimation:
reg = plm.PanelOLS.from_formula(
    formula='lwage ~ married+union + C(year)*educ + EntityEffects',
    data=wagepan, drop_absorbed=True)


results = reg.fit()

<ipython-input-20-b97fcea0d03b>:7: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ

  results = reg.fit()


In [21]:
# print regression table:
table = pd.DataFrame({'b': round(results.params, 4),
                      'se': round(results.std_errors, 4),
                      't': round(results.tstats, 4),
                      'pval': round(results.pvalues, 4)})
print(f'table: \n{table}\n')

table: 
                           b      se        t    pval
C(year)[T.1980]       1.3625  0.0162  83.9031  0.0000
C(year)[T.1981]       1.3400  0.1452   9.2307  0.0000
C(year)[T.1982]       1.3567  0.1451   9.3481  0.0000
C(year)[T.1983]       1.3729  0.1452   9.4561  0.0000
C(year)[T.1984]       1.4468  0.1452   9.9617  0.0000
C(year)[T.1985]       1.4122  0.1451   9.7315  0.0000
C(year)[T.1986]       1.4281  0.1451   9.8404  0.0000
C(year)[T.1987]       1.4529  0.1452  10.0061  0.0000
married               0.0548  0.0184   2.9773  0.0029
union                 0.0830  0.0194   4.2671  0.0000
C(year)[T.1981]:educ  0.0116  0.0123   0.9448  0.3448
C(year)[T.1982]:educ  0.0148  0.0123   1.2061  0.2279
C(year)[T.1983]:educ  0.0171  0.0123   1.3959  0.1628
C(year)[T.1984]:educ  0.0166  0.0123   1.3521  0.1764
C(year)[T.1985]:educ  0.0237  0.0123   1.9316  0.0535
C(year)[T.1986]:educ  0.0274  0.0123   2.2334  0.0256
C(year)[T.1987]:educ  0.0304  0.0123   2.4798  0.0132



## How do we interpret the results?

#### Based on the fitted model, we conclude:

#### 1. Married men earn 5% more than their single counterparts

#### 2. Union members earn 8.3% more than non-members.

#### 3. The return to education in increasingly larger accross time (1.16% in 1981, 1.48% in 1982,..., 3.04% in 1987)
